# Semiconductor Bloch Equations with Coulomb Interaction

## Abstract

This python module **SBEwithCI.py** is developed for simulating exciton effect on optical response in solid with Semiconductor Bloch Equations (SBEs). And it was used to reproduce the result in ref[<sup>1</sup>](#refer-anchor-1) to test the reliability.

## Quick Start

To use this programe, you should first create your own test py file such as **test.py** and import the **Numpy, Scipy, Math** module. 

In [9]:
import numpy as np
import scipy
from math import sqrt, log

And we also need to import the *SBE class* from **SBEwithCI** module.

In [11]:
from SBEwithCI import SBE

To make the input of parameters easy, two functions **Matplot, loadband** are provided.

In [13]:
from SBEwithCI import Matplot, loadband

Then we could start to imput the parameters for simulation, all the value in the simulation must be in atomic unit.
Following code is a simple example, please run it to check if your enviroment is correct.

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
from math import sqrt
from SBEwithCI import SBE, loadband, Matplot
import time as tp




start=tp.time()

pi=scipy.constants.pi
d=9.32
t=np.linspace(0,41*20,10000)
k=np.linspace(-pi/d, pi/d,201)    
dk=k[1]-k[0]


#set electric field 0
w1=108/27.21       #frequency
tau=41*0.2*sqrt(2) #pulse duration
fi=0               #phase
E0=2e8             #field strength
E0field=np.exp(-4*np.log(2)*((t-41*6)/tau)**2)*np.cos(w1*(t-41*6)+fi)/5.14e11*E0  #field 0

#set electric field 1 (optional)
w2=0.075         #frequency
tau1=41*2*sqrt(2) #pulse duration
fi1=0             #phase
E1=0          #field strength
delay=0          #delay between two pulses
E1field=np.exp(-4*np.log(2)*((t-41*6-delay)/tau1)**2)*np.cos(w2*(t-41*6-delay)+fi1)/5.14e11*E1 #field 1

#set band structure
VB1=np.zeros(k.size)
VB=np.array([VB1])

CB1=loadband(k, 'sio2\\CB1.txt')
CB2=loadband(k, 'sio2\\CB2.txt')
CB3=loadband(k, 'sio2\\CB3.txt')
CB4=loadband(k, 'sio2\\CB4.txt')     #load the band from txt file   
CB=np.array([CB1,CB2,CB3,CB4])/27.21 #put the band in a matrix

Ek=np.vstack((VB,CB))                #create a matrix include all the energy band




#set mapping matrix
MP=np.linspace(0,24,25,dtype=int).reshape((5,5))

#set transition dipole matrix
dipole1=np.ones(k.size)*0.1
dipole=np.array([dipole1 for i in range(25)])     #dipole should be a n^2 by k matrix ,n is the number of energy bands

dipole[[0,6,12,18,24]]=np.zeros(k.size)
dipole[[2,10,14,22,4,20,8,16]]=np.zeros(k.size)

dipole[[19,23]]=10*dipole1
dipole[[9,21]]=5*dipole1
dipole[[3,15]]=1.35*dipole1
dipole[[7,11]]=20*dipole1
dipole[[13,17]]=1*dipole1



#set dephasing time
T2=41*0.85 
T1=41*0.85


#calculate energy difference and dephasing term (for saving time) 
ddt=np.array([k*(0+0j) for i in range(25)])
for i in range(len(Ek)):
    for j in range(len(Ek)):
        if i==j:
            pass
        else:
            ddt[MP[i,j]]=Ek[j]-Ek[i]-1j/T2


#set coulomb potential
Vkq=np.ones((k.size,k.size))*(0+0j)
V0=1.8
for p in range(k.size):
    Vkq[p]=V0*dk/2/pi*2*scipy.special.kn(0,np.abs(k[p]-k))   #kn is modified Bessel function of the second kind
Vkq[np.diag_indices_from(Vkq)]=0+0j




#set filter for FFT
gauss=np.exp(-4*np.log(2)*((t-41*10)/10/41)**12)     

SBEg=SBE(d,VB,CB,ddt,dipole,k,E0field,E1field,T2,T1,t,Vkq)  #create a SBE object
SBEg.solve()                                                #solve the PDE 
SBEg.calculate(gauss)                                       #calculate the P(t),J(t),d(t),P(ω)，J(ω),d(ω), and absorption spectrum
SBEg.showEfield(rangex=(80,130))                            #plot the electric field in time and frequency, rangex is the range of energy (eV)
SBEg.showEnergy()                                           #plot all the energy bands
SBEg.plotelectrondensity(n=0,rangex=(5,7))                  #plot the electron/hole density dynamics on n-th band, rangex is the range of tiem (fs)
SBEg.plotemission(rangex=(0,600))                           #plot P(t),J(t),P(ω)，J(ω),rangex is the range of energy (eV)
SBEg.plotabsorption(rangex=(104,112),rangey=(0,-120))       #plot the absorption spectrum,rangex is the range of energy (eV)

end=tp.time()
print(end-start)

The running time for above code should be in 40s, and get the same results as shown in Figure 1.

![Figure1](absorption.png)

## Parameters Input

IMPORTANT NOTE: When you create a new array in numpy, it is a row vector.

### Load Band Structure

In the **SBEwithCI** module, the **loadband** function is provided to help you load the bandstructure from txt file. The txt file should have 2 column, first column is the k points and second one is the value of energy. The 

In [1]:
def loadband(k,filename):                                  
    band=np.loadtxt(filename).T                            #load txt data
    band[1]=savgol_filter(band[1],11,1)                    #smooth data
    f=interpolate.interp1d(band[0],band[1])                #Interpolation for your setting k grid
    newband=f(k)
    return newband

CB1=loadband(k,'bandstructure\\CB1.txt')                    #It will load the CB1 from your CB1.txt for your setting grid

After you load all the energy band respectively, you should put them in Valence/Conduct band matrix. I recommend to put them in order of energy. Then create a matrix include all the energy bands.

In [ ]:
VB=np.array([VB1,VB2,VB3,......])
CB=np.array([CB1,CB2,CB3,......])
Ek=np.vstack((VB,CB))

### Transition Dipole Matrix （TDM）

The TDM should have n*n rows and k-points columns.
And the rule for the sequence is shown in following table. 

Suppose that the system only have 3 energy bands.

| Row in TDM | Assignment |
|---|---|
|Row1|Dipole for 1st band to 1st band|
|Row2|Dipole for 1st band to 2nd band|
|Row3|Dipole for 1st band to 3rd band|
|Row4|Dipole for 2nd band to 1st band|
|Row5|Dipole for 2nd band to 2nd band|
|Row6|Dipole for 2nd band to 3rd band|
|Row7|Dipole for 3rd band to 1st band|
|Row8|Dipole for 3rd band to 2nd band|
|Row9|Dipole for 3rd band to 3rd band|



You can also use the Mapping Matrix to set the dipole. `Mapping(0,1)` is exactly equal to 1, so that `dipole[Mapping(0,1)]` represent the Row2 of TDM, here the `0` and `1` represent the first and second band.

### Energy Difference and Dephasing Term (DDT)

This term is $(\epsilon_{k}^{\lambda}-\epsilon_{k}^{\lambda^{'}}-i\frac{\hbar}{T_2})$ in the SBEs. And we calculate it in advance to saving time when solving the PDE. We can simply calculate it from `Ek` and `Mapping` by a loop. Its rule of sequence is same to TDM. The following code is a simple example.

In [ ]:
ddt=np.array([k*(0+0j) for i in range(25)])             
for i in range(len(Ek)):
    for j in range(len(Ek)):
        if i==j:
            pass
        else:
            ddt[MP[i,j]]=Ek[j]-Ek[i]-1j/T2

### Coulomb Potential

Here we employ the Coulomb interaction as 
$$V_{k,k^{'}}=V_{0}\frac{dk}{2\pi}2K_{0}(|k-k^{'}|)$$
where $V_0$ is the strength of the electron-hole interaction, and $K_0$ is the
modified Bessel function of the second kind.

The coulomb matrix could be created by a simple loop.

In [ ]:
Vkq=np.ones((k.size,k.size))*(0+0j)
V0=1.8
for p in range(k.size):
    Vkq[p]=V0*dk/2/pi*2*scipy.special.kn(0,np.abs(k[p]-k))    
Vkq[np.diag_indices_from(Vkq)]=0+0j

## SBE class

`SBEg=SBE(d,VB,CB,ddt,dipole,k,E0field,E1field,T2,T1,t,Vkq) `

As shown in above the SBE object have 12 parameters. And their form is shown in the following Table.

|Parameters|Form|
|:-:|:-:|
|lattice constant:d|constant|
|Valence Band:VB|N1 by k matrix|
|Conduct Band:CB|N2 by k matrix|
|Energy difference and dephasing term:ddt| (N1+N2)^2 by k matrix|
|Transition Dipole Moment:Dipole|(N1+N2)^2 by k matrix|
|k-points:k| 1 by k matrix|
|Electic field 0:E0field| 1 by t matrix|
|Electic field 1:E1field| 1 by t matrix|
|Coherence Dephasing Time:T2| constant|
|Density Dephasing Time:T1| constant|
|time:t|1 by t matrix|
|Coulomb Potential:Vkq| k by k matrix|

### Properties

Besides the above parameters, the SBE class also have many properties which are shown in following table.

|Variable name|Meaning|
|:-:|:-:|
|Vg|Group Velocity of all the energy bands|
|Efield|Totol electric field|
|freqs|fft of time|
|energy|freqs * 2π * 27.21 (eV)|
|Pt|Polarization|
|Pw|fft of Pt|
|SwP| \|Pw\|^2||
|Jt|Current|
|Jw|fft of Jt|
|SwJ| \|Jw\|^2|
|absorption| absorption spectrum|

All the properties can be call by 

`SBEg=SBE(d,VB,CB,ddt,dipole,k,E0field,E1field,T2,T1,t,Vkq)`

`SBEg.xxxx  (xxxx=properties' name)`

### Solving the PDE

After create a new SBE object, one should use `SBEg.solve()` to solve the PDE.

In [ ]:
    def solve(self):
        self.hindex=np.linspace(0, self.fhN-1,self.fhN,dtype=int)                         #set hole/valence band index
        self.eindex=np.linspace(self.fhN, self.feN+self.fhN-1,self.feN,dtype=int)         #set electron/conduct band index
        self.parameter1=(self.bandN,self.kN,self.T1,self.dk)                              #put all the constant parameter in a tuple
        self.parameter2=(self.time,self.Efield,self.dipole,self.ddt,self.MP,self.Vkq,self.k,self.hindex,self.eindex) #put all the array constant in a tuple
        self.sol=solve_ivp(finiteSBE, (self.time[0],self.time[-1]),self.y0, t_eval=self.time,atol=1e-10,rtol=1e-12,args=(self.parameter1,self.parameter2) #use solve_ivp to solve the PDE, change the atol and rtol to control the accuracy

The details of the SBE PDE is in `finiteSBE.py`, and it use the `numba.njit` to compile the derivative equations function to machine code.  

### Calculating the result

After solve the PDE, you should use `SBEg.calculate(filter)` to calculate the P(t),J(t),d(t),P(ω)，J(ω),d(ω), and absorption spectrum.


In [ ]:
    def calculate(self,Filter):
        y=self.sol.y.T                                            
        self.Pt=np.zeros(self.sol.t.size,dtype=complex)
        self.Jt=np.zeros(self.sol.t.size,dtype=complex)
        self.fkt=np.zeros((self.sol.t.size,self.bandN*self.kN),dtype=complex)
        self.pkt=np.zeros((self.sol.t.size,self.bandN**2*self.kN),dtype=complex)
        for i in range(len(self.sol.t)):
            yi=np.reshape(y[i],(self.bandN**2+self.bandN,self.kN))
            pk=yi[0:self.bandN**2]
            fk=yi[self.bandN**2:]
            self.Pt[i]=np.real(np.sum(np.sum(self.dipole*pk,1)))
            #self.Pt[i]=np.sum(np.sum(self.dipole*pk,1))
            self.Jt[i]=np.real(np.sum(np.sum(self.Vg*fk,1)))
            #self.Jt[i]=np.sum(np.sum(self.Vg*fk,1))
            self.pkt[i]=pk.reshape(pk.size)
            self.fkt[i]=fk.reshape(fk.size)
        self.Pw=fftshift((fft(self.Pt*Filter)))
        self.Jw=fftshift((fft(self.Jt*Filter)))
        self.SwP=np.abs(self.Pw)**2
        self.SwJ=np.abs(self.Jw)**2
        self.dt=difft(self.Pt,self.time)+self.Jt
        self.dw=fftshift((fft(self.dt*Filter)))
        self.Sw=np.abs(self.dw)**2
        self.Ew=fftshift(fft(self.E0))
        self.absorption=self.freqs*np.imag(self.Pw/self.Ew)

### Plot the result

The SBE class provide several method to help you plot your result.

In [ ]:
    def showEfield(self,rangex): #plot the electric field
        plt.figure(1)
        plt.subplot(2,1,1)
        plt.plot(self.time/41,self.Efield)
        plt.title('Electricfield')
        plt.ylabel('A.U.')
        plt.xlabel('Time(fs)')
        plt.subplot(2,1,2)
        plt.plot(self.freqs*2*pi*27.21,np.abs(fftshift(fft(self.Efield))))
        plt.xlabel('eV')
        plt.xlim(rangex[0],rangex[1])
        plt.show()

In [ ]:
    def showEnergy(self): #plot the bandstructure
        plt.figure()
        Matplot(self.Ek*27.21,self.k/pi*self.d)
        plt.title('Bandstructure')
        plt.xlabel('k(π/d)')
        plt.ylabel('energy(eV)')
        plt.figure()
        Matplot(self.Vg,self.k/pi*self.d)
        plt.title('Group Velocity')
        plt.ylabel('A.U.')
        plt.xlabel('k(π/d)')
        plt.show()

In [ ]:
    def plotelectrondensity(self,n,rangex):
        plt.figure(2)
        self.fkn=(self.fkt.T)[n*self.kN:(n+1)*self.kN]
        T,K=np.meshgrid(self.time,self.k)
        plt.contourf(T/41,K,self.fkn,cmap=cm.jet,levels=2**7);
        plt.ylabel('k')
        plt.xlabel('Time(fs)')
        plt.xlim(rangex[0],rangex[1])
        plt.colorbar()
        plt.show()

In [ ]:
    def plotcoherence(self,n,rangex):
        plt.figure(3)
        self.pkn=(self.pkt.T)[n*self.kN:(n+1)*self.kN]
        T,K=np.meshgrid(self.time,self.k)
        plt.contourf(T/41,K,self.pkn,cmap=cm.jet,levels=2**7);
        plt.ylabel('k')
        plt.xlabel('Time(fs)')
        plt.xlim(rangex[0],rangex[1])
        plt.colorbar()
        plt.show() 

In [ ]:
    def plotemission(self,rangex):
        plt.figure(3)
        plt.subplot(2,2,1)
        plt.plot(self.sol.t/41,self.Jt)
        plt.title("current")
        plt.xlabel("Time(fs)")
        plt.subplot(2,2,2)
        plt.plot(self.sol.t/41,self.Pt)
        plt.title("polarization")
        plt.xlabel("Time(fs)")
        plt.subplot(2,2,3)
        plt.semilogy(self.energy,self.SwJ)
        plt.xlim(rangex[0],rangex[1])
        plt.subplot(2,2,4)
        plt.semilogy(self.energy,self.SwP)
        plt.xlim(rangex[0],rangex[1])
        plt.legend()
        plt.show()

In [ ]:
    def plotabsorption(self,rangex,rangey):
        plt.figure(4)
        plt.plot(self.freqs*2*pi*27.21,self.absorption)
        plt.xlim(rangex[0],rangex[1])
        plt.ylim(rangey[0],rangey[1])
        plt.show()

## Delay/CEP Scan

Here I provide a example for delay scan.

In [ ]:
scanod=[]
delaytime=np.linspace(-2,5.5,21)*41
for delay in delaytime:
    w1=108/27.21
    w2=0.075
    tau=41*0.2*sqrt(2)
    fi=0
    E0=2e8 #V/cm
    E0field=np.exp(-4*np.log(2)*((t-41*6)/tau)**2)*np.cos(w1*(t-41*6)+fi)/5.14e11*E0
    tau1=41*1*sqrt(2)
    E1=1e10
    E1field=np.exp(-4*np.log(2)*((t-41*6-delay)/tau1)**2)*np.cos(w2*(t-41*6-delay)+fi)/5.14e11*E1
    SBE1=SBE(d,VB,CB,ddt,dipole,k,E0field,E1field,T2,T1,t,Vkq)  #set the SBE parameters
    SBE1.solve()         #solve the PDE cost about 40s
    SBE1.calculate(gauss)  # calculate the Pt,Jt,Pw,Jw
    SBE1.plotemission((0,20))  # plot the spectrum
    SBE1.plotabsorption((104,112),(0,-100))
    scanod.append(SBE1.absorption)
scanod=np.array(scanod)
plt.figure(5)
T,K=np.meshgrid(delaytime,SBE1.freqs)
plt.contourf(T/41,K*2*pi*27.21,np.log(-1*scanod.T),cmap=cm.jet,levels=np.linspace(1,6,2**7))   # please import cm from matplotlib
plt.ylim(102,112)
plt.colorbar() 